In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearnex.cluster import DBSCAN
from imutils.video import FileVideoStream
np.random.seed(42)
pd.options.mode.chained_assignment = None  # default='warn'
np.seterr(divide='ignore') # ignore divide by zero when calculating angle
%load_ext line_profiler

# parameters for Hough Line detection
RHO = 1                 # distance resolution in pixels of the Hough grid
THETA = np.pi / 180     # angular resolution in radians of the Hough grid
THRESHOLD = 20          # minimum number of votes (intersections in Hough grid cell)
MIN_LINE_LENGTH = 5     # minimum number of pixels making up a line
MAX_LINE_GAP = 2        # maximum gap in pixels between connectable line segments
WIDTH = 720             # width to resize the processed video to

## Optimizations
1. Use `Numpy` instead of `Pandas` for processing the centroid data
2. Use `zarr` library to write and append processed frame data to file

In [ ]:
def optimized(fname, save_video=False, savename=None, show_video=False, save_stats=False,
                    frame_limit=False):
    if savename == None:
        savename = "saber_tracking.avi"

    if save_video:
        # Initialize video writer to save the results
        out = cv2.VideoWriter(savename, cv2.VideoWriter_fourcc(*'XVID'), 30.0, 
                                 (WIDTH, WIDTH), True)

    cap = cv2.VideoCapture(fname)
    ret, frame = cap.read()
    total_frames = 500 if frame_limit else int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total=total_frames)
    frame_num = 0

    output_path = savename.replace(".avi", "_data.csv")
    # prevent appending to existing file
    if os.path.exists(output_path):
        os.remove(output_path)

    # instantiate DBSCAN for use throughout
    # n_jobs parallelisation introduces too much overhead
    db = DBSCAN(eps=5, min_samples=2)
    data = np.array([])

    while ret:
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2))
            # these channels were swapped in the notebook
            b = cv2.inRange(frame[:, :, 2], 200, 255)
            r = cv2.inRange(frame[:, :, 0], 180, 255)

            # convert to HSV for more masking options
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            v = cv2.inRange(hsv[:, :, 2], 170, 255)
            s = cv2.inRange(hsv[:, :, 1], 140, 175)

            # combine masks into one
            m1 = cv2.bitwise_and(b, s)
            m2 = cv2.bitwise_and(r, v)
            mask = cv2.bitwise_or(m1, m2)

            # Run Hough on masked image
            # Output "lines" is an array containing endpoints of detected line segments
            lines = cv2.HoughLinesP(mask, RHO, THETA, THRESHOLD, np.array([]), MIN_LINE_LENGTH, MAX_LINE_GAP)

            # process lines
            if isinstance(lines, np.ndarray):
                lines = np.squeeze(lines)
                cx = lines[:, :2].sum(axis=1).reshape(-1, 1)
                cy = lines[:, 2:].sum(axis=1).reshape(-1, 1)
                frames = np.zeros(cx.shape) + frame_num
                slopes = (lines[:, 3] - lines[:, 1]) / (lines[:, 2] - lines[:, 0])
                angles = np.rad2deg(np.arctan(slopes)).reshape(-1, 1)
                lengths = np.linalg.norm(lines[:, :2] - lines[:, 2:], axis=1).reshape(-1, 1)
                data = np.concatenate((frames, cx, cy, angles, lengths), axis=1).round(3)
                
                # filter by edge conditions and line length
                xedge_mask = np.logical_and(data[:, 1] > 100, data[:, 1] < 540)
                yedge_mask = np.logical_and(data[:, 2] > 50, data[:, 2] < 310)
                len_mask = np.logical_and(data[:, -1] > 10, data[:, -1] < 50)
                mask = np.logical_and(len_mask, xedge_mask)
                mask = np.logical_and(mask, yedge_mask)
                data = data[mask]

            # perform clustering to reduce data
            if data.size > 0:
                db.fit(data[:, 1:4])
                data = np.concatenate((data, db.labels_.reshape(-1, 1)), axis=1)
                data = data[data[:, -1] != -1]
                if data.size > 0:
                    df = df.groupby("labels", as_index=False, sort=False).mean()
                    for cx, cy in zip(df["centroid_x"], df["centroid_y"]):
                        cv2.drawMarker(frame, (int(cx), int(cy)), (0, 255, 0), markerType=cv2.MARKER_CROSS, thickness=2)

            if save_stats:
                # save or append numpy array to file
                df.to_csv(output_path, index=False, mode='a',
                            header=not os.path.exists(output_path))

                # reset data structure
                data = np.array([])

            resized = cv2.resize(frame, (WIDTH, WIDTH))

            if show_video:
                cv2.imshow("Frame", frame)
            if save_video:
                out.write(resized)
            frame_num += 1
            pbar.update(1)

            key = cv2.waitKey(1)
            if key == ord('q'):
                break
            if key == ord('p'):
                cv2.waitKey(-1) # wait until any key is pressed
            if frame_limit and frame_num == 500:
                break

    cap.release()
    if save_video:
        out.release()
    if show_video:
        cv2.destroyAllWindows()

In [ ]:
# %lprun -f optimized 
optimized("../test_video.mp4", False, "min_length_30.avi", False, True, True)

## Optimization testing

In [ ]:
lines = np.abs(np.random.normal(150, 60, size=(5,4)))

def np_line_proc(lines):
    frame_num = 0
    cx = lines[:, :2].sum(axis=1).reshape(-1, 1)
    cy = lines[:, 2:].sum(axis=1).reshape(-1, 1)
    frames = np.zeros(cx.shape) + frame_num
    slopes = (lines[:, 3] - lines[:, 1]) / (lines[:, 2] - lines[:, 0])
    angles = np.rad2deg(np.arctan(slopes)).reshape(-1, 1)
    lengths = np.linalg.norm(lines[:, :2] - lines[:, 2:], axis=1).reshape(-1, 1)
    data = np.concatenate((frames, cx, cy, angles, lengths), axis=1).round(3)
    
    # conditions
    # edge_x = 100 < centroid[0] < 540
    # edge_y = 50 < centroid[1] < 310
    # l = 50 > length > 10
    xedge_mask = np.logical_and(data[:, 1] > 100, data[:, 1] < 540)
    yedge_mask = np.logical_and(data[:, 2] > 50, data[:, 2] < 310)
    len_mask = np.logical_and(data[:, -1] > 10, data[:, -1] < 50)
    mask = np.logical_and(len_mask, xedge_mask)
    mask = np.logical_and(mask, yedge_mask)
    return data[mask]

In [ ]:
lengths = np.linalg.norm(lines[:, :2] - lines[:, 2:], axis=1).reshape(-1, 1)

@nb.njit(parallel=True)
def nb_line_proc(lines, lengths):
    frame_num = 0
    cx = lines[:, :2].sum(axis=1).reshape(-1, 1)
    cy = lines[:, 2:].sum(axis=1).reshape(-1, 1)
    frames = np.zeros(cx.shape) + frame_num
    slopes = (lines[:, 3] - lines[:, 1]) / (lines[:, 2] - lines[:, 0])
    angles = np.rad2deg(np.arctan(slopes)).reshape(-1, 1)
    data = np.concatenate((frames, cx, cy, angles, lengths), axis=1)
    
    # conditions
    # edge_x = 100 < centroid[0] < 540
    # edge_y = 50 < centroid[1] < 310
    # l = 50 > length > 10
    xedge_mask = np.logical_and(data[:, 1] > 100, data[:, 1] < 540)
    yedge_mask = np.logical_and(data[:, 2] > 50, data[:, 2] < 310)
    len_mask = np.logical_and(data[:, -1] > 10, data[:, -1] < 50)
    mask = np.logical_and(len_mask, xedge_mask)
    mask = np.logical_and(mask, yedge_mask)
    return data[mask]

_ = nb_line_proc(lines[:2], lengths[:2])

In [ ]:
def cv2_line_proc(lines):
    frame_num = 0
    cx = lines[:, :2].sum(axis=1).reshape(-1, 1)
    cy = lines[:, 2:].sum(axis=1).reshape(-1, 1)
    frames = np.zeros(cx.shape) + frame_num
    slopes = (lines[:, 3] - lines[:, 1]) / (lines[:, 2] - lines[:, 0])
    angles = np.rad2deg(np.arctan(slopes)).reshape(-1, 1)
    lengths = np.linalg.norm(lines[:, :2] - lines[:, 2:], axis=1).reshape(-1, 1)
    data = np.concatenate((frames, cx, cy, angles, lengths), axis=1).round(3)
    
    xedge_mask = cv2.inRange(data[:, 1], 100, 540)
    yedge_mask = cv2.inRange(data[:, 2], 50, 310)
    len_mask = cv2.inRange(data[:, -1], 10, 50)
    mask = cv2.bitwise_and(len_mask, xedge_mask)
    mask = np.squeeze(cv2.bitwise_and(mask, yedge_mask)) > 0
    return data[mask]

In [ ]:
%timeit np_line_proc(lines)

In [ ]:
%timeit nb_line_proc(lines, lengths)
%timeit np.linalg.norm(lines[:, :2] - lines[:, 2:], axis=1).reshape(-1, 1)

In [ ]:
%timeit cv2_line_proc(lines)

## Results
* Using OpenCV to mask line data doesn't perform much better than numpy (~35-36 microseconds)
* Tetsing `Numba` on this function requires the removal of `np.linalg.norm` from the function since they don't currently support the axis argument in this function. 
<strong>This is an open issue on their [Github](https://github.com/numba/numba/pull/7785)</strong>, but it reduces the time to ~8 microseconds even when moving the `norm` function outside the "jitted" function -> 4.5x speedup
However, this will only save 1 second when processing the full 33,838 frame video used for testing, so I'll skip for now
* 

## Examine optimized profiles

In [ ]:
def read_profile_as_df(fname):
    with open(fname, "r") as f:
        lines = f.readlines()
    arr = []
    for l in lines:
        if l.startswith("   "):
            split = [i.strip() for i in l.split("   ") if i.strip() != ""]
            if len(split) == 6:
                arr.append(split)
        elif l.startswith("Line"):
            columns = [i.strip() for i in l.split("  ") if i != ""]
    prof = pd.DataFrame(arr, columns=columns)
    # set correct data types
    prof = prof.astype({"Line #" : "uint16", "Hits" : "uint16",
            "Time" : "float64", 
             "Per Hit" : "float16",
            "% Time" : "float16"})
    return prof

In [ ]:
prof = read_profile_as_df("../profiles/optimized_unlimited_profile.txt")

top = 10
perc = 1
largest = prof["% Time"].nlargest(10)
sub = prof.iloc[largest[largest >= perc].index, :]
print(f"Of the top {top} most time-consuming lines,")
print(f"{sub.shape[0]} are >= {perc} % for {sub['% Time'].sum():.1f} % of the time taken")
sub

In [ ]:
prof = read_profile_as_df("../profiles/optimized_fvs_unlimited_profile.txt")

largest = prof["% Time"].nlargest(10)
sub = prof.iloc[largest[largest >= perc].index, :]
print(f"Of the top {top} most time-consuming lines,")
print(f"{sub.shape[0]} are >= {perc} % for {sub['% Time'].sum():.1f} % of the time taken")
sub